In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
%%html
<style>
div.output_area pre {
    white-space: pre;
}
</style>

In [2]:
import random
import seaborn as sns


In [3]:
random.seed(10)
random.randint(0, 200)

146

In [4]:
# simulate multiple cutpoints to see if all elements are equally probably to be tested 

In [3]:
import os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../lib")
from simulators import *
from benchmark import CirqCircuit, QiskitCircuit
from detectors import *

In [6]:
from math import ceil

from PIL import (
    Image,
    ImageFont,
    ImageDraw,
)

PIL_GRAYSCALE = 'L'
PIL_WIDTH_INDEX = 0
PIL_HEIGHT_INDEX = 1
COMMON_MONO_FONT_FILENAMES = [
    'DejaVuSansMono.ttf',  # Linux
    'Consolas Mono.ttf',   # MacOS, I think
    'Consola.ttf',         # Windows, I think
]


def textfile_to_image(textfile_path):
    """Convert text file to a grayscale image.

    arguments:
    textfile_path - the content of this file will be converted to an image
    font_path - path to a font file (for example impact.ttf)
    """
    # parse the file into lines stripped of whitespace on the right side
    with open(textfile_path) as f:
        lines = tuple(line.rstrip() for line in f.readlines())
        
    # choose a font (you can see more detail in the linked library on github)
    font = None
    large_font = 20  # get better resolution with larger size
    for font_filename in COMMON_MONO_FONT_FILENAMES:
        try:
            font = ImageFont.truetype(font_filename, size=large_font)
            print("-" * 80)
            print(f'IMAGE CREATION: Using font "{font_filename}".')
            print("-" * 80)
            break
        except IOError:
            print(f'Could not load font "{font_filename}".')
    if font is None:
        font = ImageFont.load_default()
        print('Using default font.')

    # make a sufficiently sized background image based on the combination of font and lines
    font_points_to_pixels = lambda pt: round(pt * 96.0 / 72)
    margin_pixels = 20

    # height of the background image
    tallest_line = max(lines, key=lambda line: font.getsize(line)[PIL_HEIGHT_INDEX])
    max_line_height = font_points_to_pixels(font.getsize(tallest_line)[PIL_HEIGHT_INDEX])
    realistic_line_height = max_line_height * 0.8  # apparently it measures a lot of space above visible content
    image_height = int(ceil(realistic_line_height * len(lines) + 2 * margin_pixels))

    # width of the background image
    widest_line = max(lines, key=lambda s: font.getsize(s)[PIL_WIDTH_INDEX])
    max_line_width = font_points_to_pixels(font.getsize(widest_line)[PIL_WIDTH_INDEX])
    image_width = int(ceil(max_line_width + (2 * margin_pixels)))

    # draw the background
    background_color = 255  # white
    image = Image.new(PIL_GRAYSCALE, (image_width, image_height), color=background_color)
    draw = ImageDraw.Draw(image)

    # draw each line of text
    font_color = 0  # black
    horizontal_position = margin_pixels
    for i, line in enumerate(lines):
        vertical_position = int(round(margin_pixels + (i * realistic_line_height)))
        draw.text((horizontal_position, vertical_position), line, fill=font_color, font=font)
    
    return image

IMAGE_FOLDER = "../data/circuit_image/famous_algos/"
image = textfile_to_image(os.path.join(IMAGE_FOLDER, 'example.txt'))
image.save(os.path.join(IMAGE_FOLDER, 'example.png'))    

--------------------------------------------------------------------------------
IMAGE CREATION: Using font "DejaVuSansMono.ttf".
--------------------------------------------------------------------------------


In [7]:
# load one qasm file
# prepare two executors 
# run the program 
# prepare KS-detector
# run detector
# get p-value

In [8]:
ROOT_QASM_FOLDER = "../data/QASMBench/famous_algos/"
algos = sorted(os.listdir(ROOT_QASM_FOLDER), reverse=True)
algos

['simon_n6.qasm',
 'seca_n11.qasm',
 'sat_n11.qasm',
 'qft_n15.qasm',
 'qaoa_n6.qasm',
 'multiply_n13.qasm',
 'multiplier_n15.qasm',
 'ising_n10.qasm',
 'dnn_n8.qasm',
 'bv_n14.qasm',
 'bb84_n8.qasm',
 'adder_n10.qasm']

In [9]:
# load one qasm file
CURRENT_FILENAME = algos[1]


In [10]:

# prepare two executors 
executors = [CirqCircuit(), QiskitCircuit()]

In [44]:
IDX_ALGO = 11

In [46]:
for CURRENT_FILENAME in algos[IDX_ALGO:IDX_ALGO+1]:
    print(CURRENT_FILENAME)
    
    QASM_FILE =  os.path.join(ROOT_QASM_FOLDER, CURRENT_FILENAME)
    with open(QASM_FILE, 'r') as f:
        qasm_content = f.read()
    print(qasm_content)

    for i, simulator in enumerate(executors): 

        # load the program
        simulator.from_qasm(qasm_content)

        # save txt and images
        filepath_txt = os.path.join(IMAGE_FOLDER, CURRENT_FILENAME + f"_{i}.txt")
        filepath_image = os.path.join(IMAGE_FOLDER, CURRENT_FILENAME + f"_{i}.png")
        out_file = open(filepath_txt, "w")
        simulator.draw(file=out_file)
        out_file.close()
        image = textfile_to_image(filepath_txt)
        image.save(filepath_image)    

adder_n10.qasm
// quantum ripple-carry adder from Cuccaro et al, quant-ph/0410184
OPENQASM 2.0;
include "qelib1.inc";

qreg cin[1];
qreg a[4];
qreg b[4];
qreg cout[1];
creg ans[5];
// set input states
x a[0]; // a = 0001
x b;    // b = 1111
// add a to b, storing result in b
cx a[0],b[0];
cx a[0],cin[0];
ccx cin[0],b[0],a[0];

cx a[1],b[1];
cx a[1],a[0];
ccx a[0],b[1],a[1];

cx a[2],b[2];
cx a[2],a[1];
ccx a[1],b[2],a[2];

cx a[3],b[3];
cx a[3],a[2];
ccx a[2],b[3],a[3];

cx a[3],cout[0];


ccx a[2],b[3],a[3];
cx a[3],a[2];
cx a[2],b[3];


ccx a[1],b[2],a[2];
cx a[2],a[1];
cx a[1],b[2];


ccx a[0],b[1],a[1];
cx a[1],a[0];
cx a[0],b[1];


ccx cin[0],b[0],a[0];
cx a[0],cin[0];
cx cin[0],b[0];

measure b[0] -> ans[0];
measure b[1] -> ans[1];
measure b[2] -> ans[2];
measure b[3] -> ans[3];
measure cout[0] -> ans[4];

               ┌────┐                                                                                                ┌─────────┐
a_0: ──────X────@───────@───X───X───@──────────

In [28]:
N_REPETITIONS = 1
stat_list = []
p_val_list = []
for _ in range(N_REPETITIONS):
    # run the program 
    results = []
    for simulator in executors:
        simulator.execute(custom_shots=8192)
        res = simulator.get_result()
        results.append(res)
    # prepare KS-detector
    detector = KS_Detector()
    # run detector
    stat, p_val = detector.check(results[0], results[1])
    # get p-value
    p_val_list.append(p_val)
    stat_list.append(stat)
print(f"Stat: {np.median(stat)}, p_val: {np.median(p_val)} on {N_REPETITIONS} repetition")

Stat: 0.01611328125, p_val: 0.23800548866747176 on 1 repetition


In [ ]:
sns.histplot(p_val_list)

# Manual Conversion with Q-convert

https://quantum-circuit.com/qconvert

In [35]:
import cirq
import numpy as np
from functools import reduce

def u3(p_theta, p_phi, p_lambda):
    return cirq.MatrixGate(np.array([[np.cos(p_theta/2), -np.exp(1j*p_lambda)*np.sin(p_theta/2)], [np.exp(1j*p_phi)*np.sin(p_theta/2), np.exp(1j*p_lambda+1j*p_phi)*np.cos(p_theta/2)]]))

q = [cirq.NamedQubit('q' + str(i)) for i in range(6)]

circuit = cirq.Circuit(
    cirq.H(q[0]),
    cirq.H(q[1]),
    cirq.H(q[2]),
    cirq.H(q[3]),
    cirq.H(q[4]),
    cirq.H(q[5]),
    cirq.rz(np.pi * -0.9153964903)(q[0]),
    cirq.rz(np.pi * -0.9153964903)(q[1]),
    cirq.rz(np.pi * -0.9153964903)(q[2]),
    cirq.rz(np.pi * -0.9153964903)(q[5]),
    u3(np.pi * 0.5, 0, 0)(q[0]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[1]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[2]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[5]),
    cirq.rx(np.pi * 0.5)(q[0]),
    cirq.CNOT(q[0], q[1]),
    cirq.rx(np.pi * 0.4153964903)(q[0]),
    cirq.ry(np.pi * 0.5)(q[1]),
    cirq.CNOT(q[1], q[0]),
    cirq.rx(np.pi * -0.5)(q[1]),
    cirq.rz(np.pi * 0.5)(q[1]),
    cirq.CNOT(q[0], q[1]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[0]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[1]),
    cirq.rz(np.pi * -0.9153964903)(q[0]),
    u3(np.pi * 0.5, 0, 0)(q[0]),
    cirq.rx(np.pi * 0.5)(q[0]),
    cirq.CNOT(q[0], q[2]),
    cirq.rx(np.pi * 0.4153964903)(q[0]),
    cirq.ry(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[2], q[0]),
    cirq.rx(np.pi * -0.5)(q[2]),
    cirq.rz(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[0], q[2]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[0]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[2]),
    cirq.rz(np.pi * -0.9153964903)(q[0]),
    u3(np.pi * 0.5, 0, 0)(q[0]),
    cirq.rx(np.pi * 0.5)(q[0]),
    cirq.CNOT(q[0], q[5]),
    cirq.rx(np.pi * 0.4153964903)(q[0]),
    cirq.ry(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[5], q[0]),
    cirq.rx(np.pi * -0.5)(q[5]),
    cirq.rz(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[0], q[5]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[0]),
    cirq.rz(np.pi * -0.9153964903)(q[1]),
    cirq.rz(np.pi * -0.9153964903)(q[2]),
    cirq.rz(np.pi * -0.9153964903)(q[3]),
    cirq.rz(np.pi * -0.9153964903)(q[4]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[5]),
    cirq.rx(np.pi * -0.6320733477)(q[0]),
    u3(np.pi * 0.5, 0, 0)(q[1]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[2]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[3]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[4]),
    cirq.rz(np.pi * -0.9153964903)(q[5]),
    cirq.rz(np.pi * 0.1487377097)(q[0]),
    cirq.rx(np.pi * 0.5)(q[1]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[5]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[0]),
    cirq.CNOT(q[1], q[2]),
    cirq.rx(np.pi * 0.5)(q[0]),
    cirq.rx(np.pi * 0.4153964903)(q[1]),
    cirq.ry(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[2], q[1]),
    cirq.rx(np.pi * -0.5)(q[2]),
    cirq.rz(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[1], q[2]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[1]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[2]),
    cirq.rz(np.pi * -0.9153964903)(q[1]),
    u3(np.pi * 0.5, 0, 0)(q[1]),
    cirq.rx(np.pi * 0.5)(q[1]),
    cirq.CNOT(q[1], q[3]),
    cirq.rx(np.pi * 0.4153964903)(q[1]),
    cirq.ry(np.pi * 0.5)(q[3]),
    cirq.CNOT(q[3], q[1]),
    cirq.rx(np.pi * -0.5)(q[3]),
    cirq.rz(np.pi * 0.5)(q[3]),
    cirq.CNOT(q[1], q[3]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[1]),
    cirq.rz(np.pi * -0.9153964903)(q[2]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[3]),
    cirq.rx(np.pi * -0.6320733477)(q[1]),
    u3(np.pi * 0.5, 0, 0)(q[2]),
    cirq.rz(np.pi * 0.1487377097)(q[1]),
    cirq.rx(np.pi * 0.5)(q[2]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[1]),
    cirq.CNOT(q[2], q[4]),
    cirq.CNOT(q[0], q[1]),
    cirq.rx(np.pi * 0.4153964903)(q[2]),
    cirq.ry(np.pi * 0.5)(q[4]),
    cirq.rx(np.pi * 0.3512622903)(q[0]),
    cirq.ry(np.pi * 0.5)(q[1]),
    cirq.CNOT(q[4], q[2]),
    cirq.CNOT(q[1], q[0]),
    cirq.rx(np.pi * -0.5)(q[4]),
    cirq.rx(np.pi * -0.5)(q[1]),
    cirq.rz(np.pi * 0.5)(q[4]),
    cirq.rz(np.pi * 0.5)(q[1]),
    cirq.CNOT(q[2], q[4]),
    cirq.CNOT(q[0], q[1]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[2]),
    cirq.rz(np.pi * -0.9153964903)(q[3]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[0]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[1]),
    cirq.rx(np.pi * -0.6320733477)(q[2]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[3]),
    cirq.rz(np.pi * -0.9153964903)(q[4]),
    cirq.rz(np.pi * 0.1487377097)(q[0]),
    cirq.rz(np.pi * 0.1487377097)(q[2]),
    u3(np.pi * 0.5, 0, 0)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[0]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[2]),
    cirq.rx(np.pi * 0.5)(q[4]),
    cirq.rx(np.pi * 0.5)(q[0]),
    cirq.CNOT(q[4], q[3]),
    cirq.CNOT(q[0], q[2]),
    cirq.ry(np.pi * 0.5)(q[3]),
    cirq.rx(np.pi * 0.4153964903)(q[4]),
    cirq.rx(np.pi * 0.3512622903)(q[0]),
    cirq.ry(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[3], q[4]),
    cirq.CNOT(q[2], q[0]),
    cirq.rx(np.pi * -0.5)(q[3]),
    cirq.rx(np.pi * -0.5)(q[2]),
    cirq.rz(np.pi * 0.5)(q[3]),
    cirq.rz(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[4], q[3]),
    cirq.CNOT(q[0], q[2]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[3]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[0]),
    cirq.rz(np.pi * 0.1487377097)(q[1]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[2]),
    cirq.rz(np.pi * -0.9153964903)(q[3]),
    cirq.rz(np.pi * -0.9153964903)(q[4]),
    cirq.rz(np.pi * 0.1487377097)(q[0]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[1]),
    cirq.rz(np.pi * 0.1487377097)(q[2]),
    u3(np.pi * 0.5, 0, 0)(q[3]),
    u3(np.pi * 0.5, 0, 0)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[0]),
    cirq.rx(np.pi * 0.5)(q[1]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[2]),
    cirq.rx(np.pi * 0.5)(q[3]),
    cirq.rx(np.pi * 0.5)(q[4]),
    cirq.rx(np.pi * 0.5)(q[0]),
    cirq.CNOT(q[1], q[2]),
    cirq.CNOT(q[4], q[5]),
    cirq.rx(np.pi * 0.3512622903)(q[1]),
    cirq.ry(np.pi * 0.5)(q[2]),
    cirq.rx(np.pi * 0.4153964903)(q[4]),
    cirq.ry(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[2], q[1]),
    cirq.CNOT(q[5], q[4]),
    cirq.rx(np.pi * -0.5)(q[2]),
    cirq.rx(np.pi * -0.5)(q[5]),
    cirq.rz(np.pi * 0.5)(q[2]),
    cirq.rz(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[1], q[2]),
    cirq.CNOT(q[4], q[5]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[1]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[2]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[4]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[5]),
    cirq.rz(np.pi * -0.9153964903)(q[5]),
    u3(np.pi * 0.5, np.pi * 1, 0)(q[5]),
    cirq.CNOT(q[3], q[5]),
    cirq.rx(np.pi * 0.4153964903)(q[3]),
    cirq.ry(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[5], q[3]),
    cirq.rx(np.pi * -0.5)(q[5]),
    cirq.rz(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[3], q[5]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, np.pi * 1)(q[3]),
    cirq.rx(np.pi * -0.6320733477)(q[4]),
    u3(np.pi * 0.5, np.pi * 0.9153964903, 0)(q[5]),
    cirq.rx(np.pi * -0.6320733477)(q[3]),
    cirq.rx(np.pi * -0.6320733477)(q[5]),
    cirq.rz(np.pi * 0.1487377097)(q[5]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[5]),
    cirq.CNOT(q[0], q[5]),
    cirq.rx(np.pi * 0.3512622903)(q[0]),
    cirq.ry(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[5], q[0]),
    cirq.rx(np.pi * -0.5)(q[5]),
    cirq.rz(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[0], q[5]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[0]),
    cirq.rz(np.pi * 0.1487377097)(q[1]),
    cirq.rz(np.pi * 0.1487377097)(q[3]),
    cirq.rz(np.pi * 0.1487377097)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[5]),
    cirq.rx(np.pi * -0.6710086873)(q[0]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[1]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[3]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[4]),
    cirq.rz(np.pi * 0.1487377097)(q[5]),
    cirq.rx(np.pi * 0.5)(q[1]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[5]),
    cirq.CNOT(q[1], q[3]),
    cirq.rx(np.pi * 0.3512622903)(q[1]),
    cirq.ry(np.pi * 0.5)(q[3]),
    cirq.CNOT(q[3], q[1]),
    cirq.rx(np.pi * -0.5)(q[3]),
    cirq.rz(np.pi * 0.5)(q[3]),
    cirq.CNOT(q[1], q[3]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[1]),
    cirq.rz(np.pi * 0.1487377097)(q[2]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[3]),
    cirq.rx(np.pi * -0.6710086873)(q[1]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[2]),
    cirq.rx(np.pi * 0.5)(q[2]),
    cirq.CNOT(q[2], q[4]),
    cirq.rx(np.pi * 0.3512622903)(q[2]),
    cirq.ry(np.pi * 0.5)(q[4]),
    cirq.CNOT(q[4], q[2]),
    cirq.rx(np.pi * -0.5)(q[4]),
    cirq.rz(np.pi * 0.5)(q[4]),
    cirq.CNOT(q[2], q[4]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[2]),
    cirq.rz(np.pi * 0.1487377097)(q[3]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[4]),
    cirq.rx(np.pi * -0.6710086873)(q[2]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[3]),
    cirq.rz(np.pi * 0.1487377097)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[4]),
    cirq.rx(np.pi * 0.5)(q[4]),
    cirq.CNOT(q[4], q[3]),
    cirq.ry(np.pi * 0.5)(q[3]),
    cirq.rx(np.pi * 0.3512622903)(q[4]),
    cirq.CNOT(q[3], q[4]),
    cirq.rx(np.pi * -0.5)(q[3]),
    cirq.rz(np.pi * 0.5)(q[3]),
    cirq.CNOT(q[4], q[3]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[3]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[4]),
    cirq.rz(np.pi * 0.1487377097)(q[3]),
    cirq.rz(np.pi * 0.1487377097)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[3]),
    u3(np.pi * 0.5, np.pi * 1.8013661765, np.pi * 1.8013661765)(q[4]),
    cirq.rx(np.pi * 0.5)(q[3]),
    cirq.rx(np.pi * 0.5)(q[4]),
    cirq.CNOT(q[4], q[5]),
    cirq.rx(np.pi * 0.3512622903)(q[4]),
    cirq.ry(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[5], q[4]),
    cirq.rx(np.pi * -0.5)(q[5]),
    cirq.rz(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[4], q[5]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[5]),
    cirq.rz(np.pi * 0.1487377097)(q[5]),
    u3(np.pi * 0.5, np.pi * 1, np.pi * 1.8013661765)(q[5]),
    cirq.CNOT(q[3], q[5]),
    cirq.rx(np.pi * 0.3512622903)(q[3]),
    cirq.ry(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[5], q[3]),
    cirq.rx(np.pi * -0.5)(q[5]),
    cirq.rz(np.pi * 0.5)(q[5]),
    cirq.CNOT(q[3], q[5]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, np.pi * 1)(q[3]),
    cirq.rx(np.pi * -0.6710086873)(q[4]),
    u3(np.pi * 0.5, np.pi * 1.0498961138, 0)(q[5]),
    cirq.rx(np.pi * -0.6710086873)(q[3]),
    cirq.rx(np.pi * -0.6710086873)(q[5]),
    cirq.measure(q[0], key='mm0'),
    cirq.measure(q[1], key='mm1'),
    cirq.measure(q[2], key='mm2'),
    cirq.measure(q[3], key='mm3'),
    cirq.measure(q[4], key='mm4'),
    cirq.measure(q[5], key='mm5')
)
print(circuit.to_text_diagram())
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1024)
result_dict = dict(result.multi_measurement_histogram(keys=['mm0', 'mm1', 'mm2', 'mm3', 'mm4', 'mm5']))
keys = list(map(lambda arr: reduce(lambda x, y: str(x) + str(y), arr[::-1]), result_dict.keys()))
counts = dict(zip(keys,[value for value in result_dict.values()]))
print(counts)

                                                                                                                                                                                                                                                                                                                                    ┌───────────┐                ┌──────────┐                          ┌──────────────────────────────┐                                                                                       ┌──────────────────────────────┐                ┌──┐                          ┌──┐                                                                                                                                                                                                                                                                                                                                                                                  ┌─────────────────────────────────────

# Inspect one 

In [2]:
QASM_PATH = "../data/experiment_v03/cross_platform/original_programs/56.qasm"
with open(QASM_PATH, 'r') as qasm_file:
    qasm_content = qasm_file.read()
    qasm_file.close()
    
print(qasm_content)

OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
creg c[6];
cx q[5], q[3];
ry(0.9801424781769557) q[0];
cx q[5], q[3];
ry(6.094123332392967) q[0];
rz(1.1424399624340646) q[1];
cx q[1], q[3];
rx(3.844385118274953) q[1];
cx q[1], q[2];
rx(1.2545873742863833) q[4];
rx(0.29185655071471744) q[3];
ry(0.4087312132537349) q[1];
U(0,5.0793103400482895,0) q[5];
cx q[0], q[4];
rx(3.1112882860657196) q[0];
cx q[5], q[1];
ry(3.267683749398383) q[1];
rx(6.092079887237463) q[1];
rz(5.622366060669442) q[5];
ry(0.55601478868818) q[5];
cx q[0], q[2];
cx q[1], q[5];
cx q[1], q[3];
cx q[4], q[0];
U(0,1.24856745052169,0) q[4];
cx q[4], q[3];
rz(0.465236267850866) q[4];
cx q[0], q[5];
ry(0.39934889266571716) q[1];
cx q[1], q[4];
ry(2.9670138796084675) q[5];
cx q[4], q[3];
rx(3.1026092358267214) q[4];
rx(0.15971308367983456) q[2];
cx q[0], q[4];
cx q[3], q[5];
cx q[2], q[4];
cx q[0], q[2];
cx q[5], q[4];
ry(5.0496205853630185) q[5];
cx q[5], q[2];
rz(1.9980747615752663) q[5];
cx q[1], q[3];
rz(0.04368152440717

In [9]:
simulator_cirq = CirqCircuit()
simulator_qiskit = QiskitCircuit()

In [10]:
simulator_cirq.from_qasm(qasm_content)
simulator_cirq.draw()

        ┌────────────────────┐               ┌─────────────────────────────────────────────────────────────────────────────────┐                    ┌──────────┐   ┌────────────┐                                                                                                       ┌──┐                                     ┌───────────┐   ┌──┐                       ┌───────────┐                                     ┌──────────────────────────────────────────────────────────────────────────────────────────┐                ┌──┐   ┌──┐   ┌──────────┐                                                                                      ┌─────────────────────────────────────────────────────────────────────────────────┐                                 ┌───────────┐   ┌─────────────────────────────────────────────────────────────────────────────────┐                ┌───────────┐   ┌─────────────────────────────────────────────────────────────────────────────────┐                                    

In [11]:
simulator_qiskit.from_qasm(qasm_content)
simulator_qiskit.draw()

     ┌─────────────┐┌────────────┐                      ┌────────────┐»
q_0: ┤ Ry(0.98014) ├┤ Ry(6.0941) ├─────────────■────────┤ Rx(3.1113) ├»
     └┬────────────┤└────────────┘             │        └────────────┘»
q_1: ─┤ Rz(1.1424) ├───────────────────────────┼──────────────■───────»
      └────────────┘                           │              │       »
q_2: ──────────────────────────────────────────┼──────────────┼───────»
                        ┌───┐     ┌───┐        │            ┌─┴─┐     »
q_3: ───────────────────┤ X ├─────┤ X ├────────┼────────────┤ X ├─────»
      ┌────────────┐    └─┬─┘     └─┬─┘      ┌─┴─┐          └───┘     »
q_4: ─┤ Rx(1.2546) ├──────┼─────────┼────────┤ X ├────────────────────»
      └────────────┘      │         │  ┌─────┴───┴─────┐              »
q_5: ─────────────────────■─────────■──┤ U(0,5.0793,0) ├──────────────»
                                       └───────────────┘              »
c: 6/═══════════════════════════════════════════════════════════